In [1]:
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder.appName("CSVtoParquet").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/26 18:43:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Leer el primer archivo CSV
df1 = spark.read \
    .format("csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("encoding", "latin1") \
    .load("/rawdata/vitales/defunciones/ConsultarDefunciones_2008_a_2016.txt")

# Leer el segundo archivo CSV
df2 = spark.read \
    .format("csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("encoding", "latin1") \
    .load("/rawdata/vitales/defunciones/ConsultarDefunciones_2017_a_2023.txt")

In [3]:
# Configurar tamaño objetivo de los archivos Parquet (aproximadamente 7MB)
target_file_size_mb = 7
target_file_size_bytes = target_file_size_mb * 1024 * 1024

# Definir una función para guardar DataFrame como Parquet con tamaño objetivo
def save_as_parquet(df, output_dir, target_size):
    # Repartir el DataFrame para aproximar el tamaño objetivo
    df = df.repartition((df.count() * df.columns.__len__() * 8) // target_size)
    # Escribir el DataFrame en el formato Parquet
    df.write.mode("overwrite").parquet(output_dir)

In [4]:
# Guardar el primer DataFrame como archivos Parquet en la carpeta especificada
save_as_parquet(df1, "/rawdata/vitales/defunciones/defunciones_parquet_2008_a_2016", target_file_size_bytes)

In [5]:
# Guardar el segundo DataFrame como archivos Parquet en la carpeta especificada
save_as_parquet(df2, "/rawdata/vitales/defunciones/defunciones_parquet_2017_a_2023", target_file_size_bytes)

In [8]:
import subprocess

# Función para renombrar archivos Parquet
def rename_parquet_files(directory):
    # Listar los archivos en el directorio
    files = subprocess.check_output(["hdfs", "dfs", "-ls", directory]).decode("utf-8").split("\n")
    parquet_files = [line.split()[-1] for line in files if line.endswith(".parquet")]
    
    # Renombrar los archivos
    for idx, file in enumerate(parquet_files):
        new_name = f"{directory}/part.{idx}.parquet"
        subprocess.call(["hdfs", "dfs", "-mv", file, new_name])

# Renombrar los archivos en las carpetas especificadas
rename_parquet_files("/rawdata/vitales/defunciones/defunciones_parquet_2008_a_2016")
rename_parquet_files("/rawdata/vitales/defunciones/defunciones_parquet_2017_a_2023")

2024-06-26 18:47:26,822 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-26 18:47:28,163 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-26 18:47:29,537 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-26 18:47:31,478 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-26 18:47:33,254 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-26 18:47:34,666 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-26 18:47:35,971 WARN util.NativeCodeLoader: Unable to load native-ha

2024-06-26 18:48:48,454 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
# Detener la sesión de Spark
spark.stop()